In [2]:
%load_ext autotime

In [3]:
import sys
sys.path.append('/Users/Pradap/Documents/Research/Python-Package/enrique/')

time: 1.16 ms


In [4]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 702 ms


In [5]:
mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')

True

time: 145 ms


In [6]:
# read walmart dataset
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv', 
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.19 s


In [7]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')

time: 88.3 ms


In [8]:
bwk.get_attr_names()

['id',
 'title',
 'author',
 'numAuthors',
 'binding',
 'publisher',
 'isbn',
 'pubYear',
 'pubMonth',
 'pubDay',
 'editionNum',
 'pages',
 'volume',
 'lang',
 'editionDescr',
 'series',
 'upc']

time: 2.52 ms


In [9]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'publisher', 'publisher', ['title', 'author'], 
                   ['title', 'author'])

time: 1min 22s


In [10]:
len(C)

18723218

time: 6.1 ms


In [11]:
feat_table = mg.get_features_for_blocking(wal, bwk)

time: 8.52 s


In [12]:
rb = mg.RuleBasedBlocker()
rb.add_rule('title_title_jac_qgm_3_qgm_3(ltuple, ltuple) < 0.8', feat_table)

True

time: 2.7 ms


In [13]:
#D = rb.block_candset(C, verbose=True, percent=0.01)

time: 756 µs


In [15]:
a = C.ix[1]

time: 1.25 ms


In [23]:
import numpy as np

time: 1.01 ms


In [25]:
C['dummy'] = np.arange(0, len(C))

time: 197 ms


In [19]:
l_df = C.get_property('ltable').to_dataframe()
r_df = C.get_property('rtable').to_dataframe()
    

time: 280 ms


In [20]:
l_df.set_index('id', inplace=True)
r_df.set_index('id', inplace=True)

time: 3.46 ms


In [26]:
def fn(t, fk_l, fk_r, ltable, rtable):
    l = t[fk_l]
    r = t[fk_r]
    c = float(t['dummy'])
    if c%10000 == 0:
        print c
    return rb.apply_rules(ltable.ix[l], rtable.ix[r])

    

time: 3.76 ms


In [ ]:
fk_l = C.get_property('foreign_key_ltable')
fk_r = C.get_property('foreign_key_rtable')

pp = C.apply(fn, axis=1, args=(fk_l, fk_r, l_df, r_df ))

0.0
10000.0
20000.0